In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             RocCurveDisplay, PrecisionRecallDisplay, ConfusionMatrixDisplay)
from sklearn.inspection import permutation_importance
import joblib, json, time

# folders
DATA_DIR = Path("../data")
MODEL_DIR = Path("../models"); MODEL_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR = Path("../dashboard/screenshots"); IMG_DIR.mkdir(parents=True, exist_ok=True)

# load engineered data from Step 7
df = pd.read_csv(DATA_DIR/"model_input.csv")
df.columns = [c.strip().lower().replace(" ","_") for c in df.columns]

# Target: use existing high_risk; else derive from score
if "high_risk" in df.columns:
    y = df["high_risk"].astype(int)
else:
    y = (df["score"] >= 1.05).astype(int)

# Feature lists (use only columns that actually exist)
num_candidates = ["score","predicted_readmission_rate","expected_readmission_rate",
                  "delta_pred_exp","ratio_pred_exp","readmit_rate_calc",
                  "number_of_discharges","score_z_condition","score_z_state"]
num_features = [c for c in num_candidates if c in df.columns]

cat_candidates = ["state","condition","measure_id","volume_band","risk_band","facility_id"]
cat_features = [c for c in cat_candidates if c in df.columns]

# Keep a few ID cols to attach to predictions later
id_cols = [c for c in ["facility_id","state","condition","measure_id"] if c in df.columns]

X = df[num_features + cat_features].copy()
mask = y.notna()
X, y = X[mask], y[mask]
X.shape, y.mean()  # class balance check


((18510, 15), np.float64(0.6443544030253917))

In [2]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# preprocessing
numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median")),
                       ("scaler", StandardScaler())])
categorical_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                           ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocess = ColumnTransformer([
    ("num", numeric_tf, num_features),
    ("cat", categorical_tf, cat_features)
])

models = {
    "log_reg": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "rf": RandomForestClassifier(n_estimators=300, random_state=42,
                                 class_weight="balanced_subsample")
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = {}
for name, model in models.items():
    pipe = Pipeline([("pre", preprocess), ("clf", model)])
    scores = cross_val_score(pipe, X_train, y_train, scoring="roc_auc", cv=cv)
    cv_scores[name] = (scores.mean(), scores.std())

cv_scores  # pick the higher mean ROC-AUC


{'log_reg': (np.float64(0.9998046909996596),
  np.float64(0.00039061800068092013)),
 'rf': (np.float64(1.0), np.float64(0.0))}

In [3]:
best_name = max(cv_scores, key=lambda k: cv_scores[k][0])
best_model = models[best_name]

pipe = Pipeline([("pre", preprocess), ("clf", best_model)])
pipe.fit(X_train, y_train)

y_proba = pipe.predict_proba(X_test)[:, 1]
y_pred  = (y_proba >= 0.5).astype(int)

roc = roc_auc_score(y_test, y_proba)
print(f"Best model: {best_name} | ROC-AUC: {roc:.3f}")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot()
plt.title("Confusion Matrix")
plt.savefig(IMG_DIR/"cm.png", dpi=160, bbox_inches="tight"); plt.close()

# ROC + PR curves
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("ROC Curve")
plt.savefig(IMG_DIR/"roc_curve.png", dpi=160, bbox_inches="tight"); plt.close()

PrecisionRecallDisplay.from_predictions(y_test, y_proba)
plt.title("Precision–Recall Curve")
plt.savefig(IMG_DIR/"pr_curve.png", dpi=160, bbox_inches="tight"); plt.close()


Best model: rf | ROC-AUC: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1317
           1       1.00      1.00      1.00      2385

    accuracy                           1.00      3702
   macro avg       1.00      1.00      1.00      3702
weighted avg       1.00      1.00      1.00      3702



In [5]:
# Cell 4 — Permutation importance with reliable feature names
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Compute importances on the test set
r = permutation_importance(
    pipe, X_test, y_test,
    n_repeats=10, random_state=42, scoring="roc_auc"
)

# 2) Get transformed feature names aligned to what the model sees
pre = pipe.named_steps["pre"]

def get_transformed_feature_names(preprocessor, num_feats, cat_feats):
    # Try the modern API first (works on recent scikit-learn)
    try:
        return preprocessor.get_feature_names_out()
    except Exception:
        pass
    # Fallback: build names manually for numeric + OHE categorical
    cat_names = np.array([])
    try:
        cat = preprocessor.named_transformers_.get("cat")
        if cat is not None and hasattr(cat.named_steps["onehot"], "get_feature_names_out") and cat_feats:
            cat_names = cat.named_steps["onehot"].get_feature_names_out(cat_feats)
    except Exception:
        pass
    num_names = np.array(num_feats, dtype=object)
    return np.concatenate([num_names, cat_names])

feat_names = get_transformed_feature_names(pre, num_features, cat_features)

# 3) Final safety: trim to the same length if anything is off
n = min(len(feat_names), r.importances_mean.shape[0])
feat_names = np.asarray(feat_names)[:n]
importances = r.importances_mean[:n]

# 4) Plot & save
imp = pd.DataFrame({"feature": feat_names, "importance": importances}) \
        .sort_values("importance", ascending=False).head(20)

ax = imp.plot.barh(x="feature", y="importance", figsize=(8,8))
ax.invert_yaxis()
plt.title("Top 20 Features (Permutation Importance)")
plt.tight_layout()
plt.savefig(IMG_DIR / "feature_importance.png", dpi=160, bbox_inches="tight")
plt.close()

imp.head(10)


,feature,importance
2,num__expected_readmission_rate,3.330669e-17
0,num__score,1.110223e-17
1,num__predicted_readmission_rate,1.110223e-17
3,num__delta_pred_exp,1.110223e-17
5,num__readmit_rate_calc,1.110223e-17
6,num__number_of_discharges,1.110223e-17
7,num__score_z_condition,1.110223e-17
10,cat__state_AL,1.110223e-17
11,cat__state_AR,1.110223e-17
14,cat__state_CO,1.110223e-17


In [6]:
# Save pipeline
model_path = MODEL_DIR/f"{best_name}_pipeline.joblib"
joblib.dump(pipe, model_path)

# Save metrics
metrics = {
    "best_model": best_name,
    "roc_auc": float(roc),
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
}
with open(MODEL_DIR/"metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Export test predictions with IDs for BI
preds = pd.DataFrame(index=X_test.index)
for col in id_cols:
    if col in df.columns:
        preds[col] = df.loc[preds.index, col]
preds["y_true"]  = y_test.values
preds["y_proba"] = y_proba
preds["y_pred"]  = y_pred

preds_path = DATA_DIR/"predictions_test.csv"
preds.to_csv(preds_path, index=False)

model_path, preds_path, metrics


(PosixPath('../models/rf_pipeline.joblib'),
 PosixPath('../data/predictions_test.csv'),
 {'best_model': 'rf', 'roc_auc': 1.0, 'timestamp': '2025-08-15 13:41:06'})